In [1]:
import pandas as pd
import re
from random import randint
import spacy
from time import time

# Folketingstidende datawrangling

Det følgende script er en renskrivning af et "trial and error" script jeg tidligere har udarbejdet. Dette script kan findes på min github:

### Formål
Dette script ønsker at formatere "folketingstidende" på en sådan måde at der kan laves en digitalhistorisk analyse af data. Formålet med scriptet er da at "genkende" når en ny person begynder at tale fra folketingsets talerstol, så folektingstidende kan opdeles i folketingstaler og udtræke metadateen "hvem der taler" samt "hvilket parti tilhører taleren". Derudover skal scriptet søge at genkende når der er "fejl" i data som skal sorteres fra. Her er primært tale om at sortere marginer fra. Da der er tale om en "OCR" læsning af folketingstidende starter hver side med marginen på siden, som hvis denne ikke sorteres fra, vil indgå som en del af et folketingsmedlem eller ministers tale. 

### Genkendelse af talere:

I det følgende vil jeg søge at benytte mig af regex til at genkende når en ny person indtager folketingsets talerstol. Til dette vil jeg udnytte hvordan folketingstidende er "formateret" af udgiver. Når en ny person indtager folketingets talerstol vil dette blive skrevet som "Navn på talende folketingsmedlem (Parti som folketingsmedlem tilhører)". Det er altså dette jeg vil søge at genkende.

Jeg starter med at indlæse en del af folketingstidende som prøve.

In [3]:
df = pd.read_csv('../../Folketingstidende/Hele/19801.csv',sep=';', encoding = "latin1")

Jeg finder ny søjlenavnene:

In [3]:
df.columns.values.tolist()

['Unnamed: 0', 'spalte', 'TextBlock', 'TopMargin', 'indhold', 'år', 'samling']

Det er søjlen "indhold" der primært har interesse.

In [4]:
FT_data = [indhold for indhold in df["indhold"]]

In [8]:
FT_data[0:10]

['(1)',
 '1',
 '7/10 80: Folketingets åbning',
 '2',
 '1. møde',
 "Tirsdag den 7. oktober 1'980 kl. 12.00",
 'Aldersformanden (Ninn-Hansen):',
 'I henhold til grundloven er folketinget i dag trådt sammen til sit første møde i det nye folketingsår.',
 'Som det medlem, der længst har haft sæde i tinget, påhviler det mig at lede valget af tingets præsidium og tingsekretærer.',
 'Til formand har en gruppe af tingets medlemmer omfattende socialdemokratiets, venstres, det konservative folkepartis, socialistisk folkepartis, det radikale venstres, centrumdemokraternes og kristeligt folkepartis medlemmer indstillet den hidtidige formand, K. B. Andersen (S). Der foreligger ikke andre indstillinger. Hvis ingen begærer afstemning, vil jeg betragte indstillingen som vedtaget. (Ophold). Den er vedtaget.']

Her viser jeg de første 10 OCR-læste afsnit i folketingstidende.

Jeg vil nu søge at genkende når der er folk der taler fra folketingsets talerstol.

In [100]:
Prøve_FT = []
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"] #liste af partier
for sentence in FT_data:
    #if len(re.findall("(\w{1,10}\([A-Z]+\)|\(KrF\))", sentence)) > 0:
    #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) > 0:
    if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0:
        if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
            if len(re.findall("Ny stedf.", sentence)) > 0:
                pass
            elif len(re.findall("Udtræder:", sentence)) > 0:
                pass
            elif len(re.findall("Nyt medl", sentence)) > 0:
                pass
            elif len(re.findall(" af", sentence.lower())) > 0:
                pass
            elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                Prøve_FT.append(sentence)

Jeg kan nu undersøges hvad der genkendes:

In [108]:
Prøve_FT.sort(key = len, reverse = True)

In [109]:
len(Prøve_FT)

9346

In [110]:
Prøve_FT

['I overenstemmelse med en henstilling, som jeg har modtaget fra kulturudvalget, hvortil er henvist forslag til folketingsbeslutning vedrørende videokassettebånd med voldsfilm [af Poul H. Møller (KrF) m. fl.] (B 75), skal jeg foreslå, at det nævnte forslag i stedet henvises til retsudvalget. Hvis ingen gør indsigelse, vil jeg betragte dette forslag som vedtaget. (Ophold). Det er vedtaget.',
 'Johan Philipsen (V): »Meningen med, at vi nu får et stort indvandsreservoir med fladt vand, er jo, at vi får en enestående lejlighed til at lave et fuglereservat i dette land.«',
 'Forslag til folketingsbeslutning vedrørende videokassettebånd med voldsfilm [af Poul H. Møller (KrF) m. fl.].',
 'Forslag til folketingsbeslutning om forbud mod reklamering for stærke drikke [af Inge Krogh (KrF) m. fl.].',
 'Mørch (S) (se foran) ? vedtoges med 82 stemmer mod 56; 5 stemte hverken for eller imod.',
 'Forslag til folketingsbeslutning om reagensglasbørn [af Inge Krogh (KrF) m. fl.].',
 'Forslag til folketin

In [103]:
sentence_p = 'I overenstemmelse med en henstilling, som jeg har modtaget fra kulturudvalget, hvortil er henvist forslag til folketingsbeslutning vedrørende videokassettebånd med voldsfilm [af Poul H. Møller (KrF) m. fl.] (B 75), skal jeg foreslå, at det nævnte forslag i stedet henvises til retsudvalget. Hvis ingen gør indsigelse, vil jeg betragte dette forslag som vedtaget. (Ophold). Det er vedtaget.'

In [112]:
re.findall("(\A.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence_p)

['videokassettebånd med voldsfilm [af Poul H. Møller (KrF)']

Det ovenstående søger at genkende når der er et menigt medlem(hermed ment ikke formanden eller en minister) som ønsker at tale. Problemet er her at opremsningen af medlemmer af folketinget i starten af folketingsåret genkendes som "en taler" samt at metatale lavet af formanden genkendes eks. "Medlem af folketinget Askjær Jørgensen (FP) har meddelt mig, at han ønsker til industriministeren at stille følgende:". I begge tilfælde ser jeg det dog som mindre problematisk. 
I første tilfælde vil de "genkendte" talere blive tildelt en "tale" som er tom, altså at disse ikke siger noget, hvilket vil blive sorteret fra når jeg udtrækker de folketingstaler jeg ønsker at analysere. 
I andet tilfælde af problemet er ligeledes mindre problematisk. Her er der tale om at folketingets formand fremsætter et menigt medlems holdninger, og deraf ser jeg det ikke problematisk at "karakterisere" disse som folketingstaler.

### Genkendelse af ministre:

In [117]:
ministre_FT = []
for sentence in FT_data:
    if len(sentence) < 100:
        if len(re.findall("\w*minister[\w ]*\([\w ]*\)", sentence.lower())) > 0:
            ministre_FT.append(sentence)

In [118]:
ministre_FT.sort(key = len, reverse = True)

In [119]:
ministre_FT

['Administrator Bent Collin, Nationalmuseet, repræsentant for kulturministeriet (formand)',
 "Ministeren for kulturelle anliggender (Lise Østergaard): \\'",
 'Ministeren for offentlige arbejder (Jens Risgaard Knudsen):',
 'Ministeren for offentlige arbejder (Jens Risgaard Knudsen):',
 'Ministeren for kulturelle anliggender (Lise 0stergaard) :',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Li

Det ovenstående genkender meget godt når der er tale om en minister.

### Genkendelse af Formanden:

In [120]:
count_formanden = 0
for sentence in FT_data:
    if sentence.lower() == "formanden" or sentence.lower() == "formanden:":
        count_formanden += 1
count_formanden

1024

Jeg undersøger nu om der kan være nogle datapunkter der er blevet misset:

In [122]:
formanden_problem = []
for sentence in FT_data:
    if len(sentence) <= 15:
        if len(re.findall("formanden", sentence.lower())) > 0:
            if sentence.lower() != "formanden" and sentence.lower() != "formanden:":
                formanden_problem.append(sentence)
formanden_problem

['[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '(Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '.. Formanden:',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[For

I det ovenstående er der tale om at der står i margenen af folketingstidende, at det er formanden der taler, det er derfor ikke problematisk at disse sorteres fra.

### Marginer sorteres fra:

In [125]:
for sentence in FT_data:
    if len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
        if len(sentence) < 200:
             print(sentence)

11 7/10 80: Statsministerens åbningstale 12
15 7/10 80: Statsministerens åbningstale 16
17 7/10 80: Statsministerens åbningstale IR
19 7/10 80: Statsministerens åbningstale 20
21 7/10 80: Statsministerens åbningstale. Skr. del 22
23 7/10 80: Statsministerens åbningstale. Skr. del 24
35 7/10 80: Statsministerens åbningstale. Skr. del 36
39 7/10 80: Statsministerens åbningstale. Skr. del 40
41 7/10 80: Statsministerens åbningstale. Skr. del 11 42
43 7/10 80: Statsministerens åbningstale. Skr. del 44
45 7/10 80: Statsministerens åbningstale. Skr. del 46
47 7/10 80: Statsministerens åbningstale. Skr. del 48
49 7/10 80: Statsministerens âbningstale. Skr. del 50
51 7/10 80: Statsministerens åbningstàle. Skr. del 52
53 7/10 80: Statsministerens åbningstale. Skr. del 54
55 7/10 80: Statsministerens åbningstale. Skr. del 56
57 7/10 80: Meddelelser fra formanden - 58
59 7/10 80: Skriftlig fremsættelse 60
61 8/10 80: Dagsorden - 62
63 8/10 80: Meddelelser fra formanden 64
71 8/10 80: Meddelelse o

3259 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3260
3267 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3268
3271 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3272
3275 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet
3279 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3280
3283 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet
3285 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3286
3287 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3288
3291 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet
3295 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3296
3299 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3300
3313 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3314
3315 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet
3319 28/11 80: Forespørgsel om nedskæringer på uddannelsesområdet 3320
3323 28/11 80: Forespørgse

5133 21/1 81: Dagsorden 5134
5135 21/1 81: Spørgsmål til ministrene 5136
5137 21/1 81: Spørgsmål til ministrene 5138
5139 21/1 8 : Spørgsmål til ministrene 5 1 40
5141 21/1 81: Spørgsmål til ministrene
5143 21/1 81: Spørgsmål til ministrene 5144
5145 21/1 81: Spørgsmål til ministrene 5146
5147 21/1 81: Spørgsmål til ministrene
5149 21/1 81: Spørgsmål til ministrene
5151 21/1 81: Spørgsmål til ministrene
5155 21/1 81: Spørgsmål til ministrene 5156
5157 21/1 81: Spørgsmål til ministrene
5159 21/1 81: Spørgsmål til ministrene 5160
5163 21/1 81: Spørgsmål til ministrene 5164
5167 21/1 81: Spørgsmål til ministrene 5168
5169 21/1 81: Spørgsmål til ministrene
5171 21/1 81: Skriftligt besvarede spørgsmål 5172
5173 21/181: Skriftligt besvarede spørgsmål
5175 21/1 81: Skriftligt besvarede spørgsmål 5176
5177 21/1 81: Skriftligt besvarede spørgsmål
5179 21/1 81: Skriftligt besvarede spørgsmål 5180
5181 21/1 81: Skriftligt besvarede spørgsmål 5182
5183 21/1 81: Skriftligt besvarede spørgsmål 5184


7143 24/2 81 :cJForhandling om redegørelse med baggrund i telerapporten
7145 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7146
7147 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7148
7149 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7150
7151 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7152
7153 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7154
7155 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7156
7157 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7158
7159 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7160
7161 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7162
7163 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7164
7165 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7166
7167 24/2 81: Forhandling om redegørelse med baggrund i telerapporten 7168
7175 25/2 81: Spørgsmål til 

9589 8/4 81: Skriftligt besvarede spørgsmål 9590
9591 8/4 81: Skriftligt besvarede spørgsmål 9592
9595 8/4 81:1. beh. af f. t. beslutn. om at gå imod neutronbomben 9596
9599 8/4 81:1. beh. af f. t. beslutn. om at gå imod neutronbomben 9600
9611 8/4 81: l. beh. af f. t. beslutn. om at gå imod neutronbomben 9612
9613 8/4 81: l. beh. af f. t. beslutn. om at gå imod neutronbomben 9614
9625 8/4 81: l. beh. af f. t. beslutn. vedr. offentlige veje 9626
9627 8/4 81: l. beh. af f. t. beslutn. vedr. offentlige veje 9628
9629 8/4 81: 1. beh. af f. t. beslutn. vedr. offentlige veje 9630
9631 8/4 81: l. beh. af f. t. beslutn. vedr. offentlige veje 9632
9641 8/4 81: l. beh. af f. t. beslutn. om forbedring af den kollektive trafik 9642
9643 8/4 81: l. beh. af f. t. beslutn. om forbedring af den kollektive trafik 9644
9645 8/4 81: l. beh. af f. t. beslutn. om forbedring af den kollektive trafik 9646
9647 8/4 81: l. beh. af f. t. beslutn. om forbedring af den kollektive trafik 9648
9655 8/4 81: l. beh.

12543 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen
12547 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen
12549 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen 12550
12551 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen 12552
12553 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen 12554
12555 2/6 81:2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen 12556
12559 2/6 81:2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen 12560
12563 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen 12564
12567 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Skt. Lukas Stiftelsen og Diakonissestiftelsen
12571 2/6 81: 2. (sidste) beh. af f. t. beslutn. vedr. Sk

Det er klart at det ovenstående er marginerne på hver side. Hvis man dog følger tallet til venstre, som er det første spalte(af to) på hver side af folketingstidende, kan man dog se at det ikke er alle marginer der findes her.

In [130]:
for sentence in FT_data:
    if len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                 print(sentence)

7/10 80: Folketingets åbning
7/10 80: Statsministerens åbningstale
7/10 80: Statsministerens åbningstale
7/10 80: Statsministerens åbningstale
7/10 80: Statsministerens åbningstale
7/10 80: Statsministerens åbningstale
7/10 80: Statsministerens åbnìngstale. Skr. del 26
7/10 80: Statsministerens åbningstale. Skr. del 28
7/10 80: Statsministerens åbningstale. Skr. del
7/10 80: Statsministerens åbningstale. Skr. del
8/10 80: Skriftlig fremsættelse 68
8/10 80: Spørgsmål til ministrene
9/10 80: Dagsorden
9/10 80: Meddelelse af orlov
7/10 80).
9/10 80: Åbningsdebat
9/10 80: Ábningsdebat 148
9/10 80: Åbningsdebat 168
9/10 80: Àbningsdebat
9/10 80: Àbningsdebat
9/10 80: Åbningsdebat 1R4
9/10 80: Åbningsdebat
9/10 80: Åbningsdebat 204
9/10 80: Åbningsdebat
9/10 80: Åbningsdebat
9/10 80: Abningsdebat
9/10 80: Åbningsdebat
9/10 80: Åbningsdebat
9/10 80: Åbningsdebat
9/10 80: Abningsdebat
9/10 80: Åbningsdebat 240
9/10 80: Ábningsdebat
9/10 80: Åbningsdebat
9/10 80: Åbningsdebat
9/10 80: Ábningsde

26/11 80: Skriftligt besvarede spørgsmål
26/11 80: Skriftligt besvarede spørgsmål
26/11 80: Skriftligt besvarede spørgsmål
26/11 80: Skriftligt besvarede spørgsmål
26/11 80: Skriftligt besvarede spørgsmål
26/11 80: Skriftligt besvarede spørgsmål
26/11 80: Skriftligt besvarede spørgsmål 2974
26/11 80: l. beh. af f. t. 1. om miljøstøtte
26/11 80: l. beh. af f. t. I. om miljøstøtte
26/11 80: 1. beh. af f. t. I. om miljøstøtte
26/11 80: l. beh. af f. t. I. om miljøstøtte
26/11 80: l. beh. af f. t. I. om miljøstøtte
26/11 80: 1. beh. af f. t. I. om miljøstøtte
26/11 80: l. beh. af f. t. I. vedr. okkergener i vandløb
26/11 80: l. beh. af f. t. l. vedr. okkergener i vandløb
26/11 80: l. beh. af f. t. l. vedr. okkergener i vandløb
26/11 80: l. beh. af f. t. l. vedr. okkergene( i vandløb
26/11 80: 1. beh. af f. t. 1. vedr. okkergener i vandløb
26/11 80: l. beh. af f. t. beslutn. om udtagelse til nævninge- oe domsmandssrnndtistf-n
26/11 80: l. beh. af f. t. beslutn. om udtagelse til nævninge- og

6/2 81: Meddelelser fra formanden
11/2 81: Dagsorden
11/2 81: Skriftlig fremsættelse
11/2 81: Skriftlig fremsættelse
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/281: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene 6346
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene 6356
11/2 81 : Spørgsmål til ministrene
11/2 81: Spørgsmål til ministrene
11/2 81: Skriftligt besvarede spørgsmål
11/2 81: Skriftligt besvarede spørgsmål 6382
11/2 81: Skriftligt besvarede spørgsmål
11/2 81: Skriftligt besvarede spørgsmål
11/2 81: Skriftligt besvarede spørgsmål
11/2 81: Skriftligt besvarede spørgsmål
11/2 81: Skriftl

26/3 81: 1. beh. af f. t. 1. om visse havanlæg
26/3 81: I. beh. af f. t. I. om visse havanlæg
26/3 81: l. beh. af f. t. 1. om visse havanlæg
26/3 81: l. beh. af f. t. l. om visse havanlæg
26/3 81: l. beh. af f. t. l. om visse havanlæg
26/3 81: I. beh. af f. t. I. om visse havanlæg
27/3 81: Afgør. af forsl. om udvalgshenv af f. t. 1. om statsovertagelse m. m. af jord
27/3 8 I: 3. beh. af f. t. I. vedr. beskatningen af pensionsordninger m. v.
27/3 81: 3. beh. af f. t. l. vedr. landsbrugsejendomme
27/3 81: 3. beh. af f. t. l. vedr. landsbrugsejendomme
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova 8870
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgsel om Cheminova
27/3 81: Forespørgse

7/5 81. Meddelelser fra formanden
8/5 81: Dagsorden
8/5 81: Meddelelser fra formanden
8/5 81: Meddelelse af orlov
8/5 81: 3. beh. af f. t. I. vedr. anbringelse m.m. af umyndiges midler m.v.
8/5 81: 2. beh. af f. t. l. vedr. særlig indkomstskat m.v. 10830
8/5 81: l. beh. af f. t. beslutn. om forbud vedr. olieprodukter til Sydafrika
8/5 81: l. beh. af f. t. beslutn. om forbud vedr. olieprodukter til Sydafrika
8/5 81: l. beh. af f. t. beslutn. om forbud vedr. olieprodukter til Sydafrika
8/5 81: l. beh. af f. t. beslutn. om forbud vedr. olieprodukter til Sydafrika
8/5 81: l. beh. af f. t. beslutn. om forbud vedr. olieprodukter til Sydafrika
8/5 81: l. beh. af f. t. beslutn. om forbud vedr. olieprodukter til Sydafrika
8/5 81: l. beh. af f. t. beslutn. om forbud vedr. olieprodukter -til Sydafrika
8/5 81:1. beh. af f. t. beslutn. om forbud vedr. olieprodukter til Sydafrika
8/5 81: l. beh. af f. t. beslutn. om forbud vedr. olieprodukter til Sydafrika
8/5 81: l. beh. af f. t. beslutn. om forbud

Her kan det også ses at der er tale om marginer. Disse skal derfor ligeledes sorteres fra. Problemet er her at spaltenumrene da er sorteret som individuelle datapunkter. Jeg prøver da også at sortere disse fra.

In [136]:
for sentence in FT_data:
    if re.fullmatch("[1-9]{1,10}", sentence.lower()): #tilføjet til at fange headers på sider
        print(sentence)

1
2
3
4
5
6
7
8
9
13
14
25
27
29
31
37
38
65
67
69
72
74
117
118
119
127
128
136
139
147
167
177
178
179
182
183
199
212
218
219
227
228
229
231
232
233
234
235
236
237
238
239
243
244
245
246
247
248
249
251
252
259
262
264
265
267
268
271
287
288
291
292
295
312
355
397
398
439
627
694
695
818
819
915
919
947
951
1122
1147
1171
1199
1223
1283
1294
1298
1311
1333
1334
1335
1336
1342
1343
1345
1348
1353
1354
1356
1357
1361
1369
1372
1373
1386
1387
1388
1389
1413
1417
1421
1425
1427
1429
1431
1432
1433
1434
1435
1436
1437
1438
1441
1444
1445
1449
1452
1456
1457
1468
1469
1472
1477
1478
1481
1484
1485
1486
1488
1512
1513
1517
1548
1555
1577
1596
1615
1617
1631
1657
1658
1661
1665
1669
1677
1678
1679
1684
1685
1689
1719
1725
1728
1738
1739
1742
1743
1747
1764
1775
1777
1779
1783
1787
1788
1798
1814
1815
1833
1836
1842
1843
1844
1845
1846
1847
1848
1849
1853
1859
1868
1871
1874
1875
1878
1879
1885
1895
1897
1911
1915
1916
1918
1919
1927
1929
1932
1935
1942
1943
1951
1971
2121
2127
2128
213

8868
8869
8872
8873
8874
8875
8876
8877
8878
8881
8882
8883
8884
8885
8886
8887
8888
8889
8891
8892
8893
8894
8895
8896
8898
8912
8913
8914
8915
8916
8917
8921
8922
8924
8925
8926
8927
8928
8931
8932
8933
8934
8935
8936
8937
8938
8939
8941
8944
8945
8946
8947
8948
8949
8951
8952
8953
8956
8962
8963
8964
8965
8967
8968
8977
8978
8979
8981
8982
8984
8985
8991
8995
8996
9111
9112
9113
9114
9115
9116
9117
9118
9119
9121
9122
9123
9124
9125
9126
9127
9128
9129
9131
9132
9133
9134
9135
9136
9137
9138
9139
9141
9142
9143
9144
9145
9146
9147
9148
9149
9151
9152
9153
9154
9155
9156
9157
9158
9159
9161
9162
9163
9164
9165
9166
9167
9168
9169
9171
9172
9173
9174
9175
9176
9177
9178
9179
9181
9182
9183
9184
9185
9186
9187
9188
9189
9191
9192
9193
9194
9195
9196
9197
9198
9199
9211
9212
9213
9214
9215
9216
9217
9218
9219
9221
9222
9223
9224
9225
9226
9227
9228
9229
9231
9232
9233
9234
9235
9236
9237
9238
9239
9241
9242
9243
9244
9245
9246
9247
9248
9249
9251
9252
9253
9254
9255
9256
9257
9258
9259


Hvis man kigger tallene igennem er det meget plausibelt at der her er tale om de manglende spaltenumre. Disse skal altså også sorteres fra.

I marginerne optræder altså også en navn på vedkommende der er den "tallende" når der skiftes spalte. Disse vil jeg også forsøge at sortere fra, da jeg ikke ønsker at en taler fra folketingets talerstols eget navn skal indgå i dennes tale. Heldigvis har disse navne også en relativt nem formatering, nemlig \[navn\]. Jeg søger altså efter datapunker der matcher denne formatering.

In [150]:
for sentence in FT_data:
    if re.fullmatch("\[.{1,100}\]", sentence.lower()):
        print(sentence)

[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Statsministeren]
[Finansministeren]
[Formanden]
[Boligministeren]
[Boligministeren]
[Boligministeren]
[Formanden]
[Finansministeren]
[Finansministeren]
[Ministeren for offentlige arbejder]
[Ministeren for offentlige arbejder]
[Ministeren for offentlige arbejder]
[Brøndsted]
[Grove]
[Nathalie Lind]
[Nathalie Lind]
[Ellemann-Jensen]
[Anders Poulsen]
[Anders Poulsen]
[Indenrigsministeren]
[Steffensen]
[Ministeren for skatter og afgifter]
[Ministeren for skatter og afgifter]
[Ministeren for skatter og afgifter]
[Tastesen]
[Tastes

[Lilli Gyldenkilde]
[Lilli Gyldenkilde]
[Niels Helveg Petersen]
[Tyroll]
[Tyroll]
[Tyroll]
[Chr. Christensen]
[Ib Christensen]
[Statsministeren]
[Lilli. Gyldenkilde]
[Udenrigsministeren]
[Steen Folke]
[Steen Folke]
[Energiministeren]
[Energiministeren]
[Kalnæs]
[Energiministeren]
[Nymann]
[Poulsgaard]
[Poulsgaard]
[Dyremose]
[Socialministeren]
[Halvgaard]
[Halvgaard]
[Kalnæs]
[Keld Andersen]
[Keld Andersen]
[Indenrigsministeren]
[Arbejdsministeren]
[Maisted]
[Miljøministeren]
[Socialministeren]
[Glistrup]
[Udenrigsministeren]
[Justitsministeren]
[Bolvig]
[Udenrigsministeren]
[Steenholdt]
[Askjær Jørgensen]
[Miljøministeren]
[Glistrup]
[Glistrup]
[Ministeren for skatter og afgifter]
[Arbejdsministeren]
[Svend Andersen]
[Sønderby]
[Ole Bernt Henriksen]
[Thorndahl]
[Dræbye]
[Waldorff]
[Waldorff]
[Miljøministeren]
[Lissa Mathiasen]
[Ole Bernt Henriksen]
[Rahbæk Møller]
[Keld Albrechtsen]
[Fogh Rasmussen]
[Majken Hessner ]
[Kalnæs]
[Birgith Mogensen]
[Fogh Rasmussen]
[Majken Hessner]
[Majke

[Palle Simonsen]
[Palle Simonsen]
[Glensgård]
[Glensgård]
[Glensgård]
[Ingerlise Koefoed]
[Ingerlise Koefoed]
[lngerlise Koefoed]
[Bilgrav-Nielsen]
[Bilgrav-Nielsen]
[Bilgrav-Nielsen]
[Mimi Jakobsen]
[Mimi Jakobsen]
[Kurt Hansen]
[Kurt Hansen]
[Kurt Hansen]
[Formanden]
[Chr. Christensen]
[Knud Lind]
[Ministeren for offentlige arbejder]
[Undervisningsministeren]
[Undervisningsministeren]
[Knud Lind]
[Justitsministeren]
[Justitsministeren]
[Socialministeren]
[Annelise Gotfredsen]
[Ministeren for kulturelle anliggender]
[Annelise Gotfredsen]
[Energiministeren]
[Wilhjelm]
[Energiministeren]
[Undervisliingsministeren]
[Undervisningsministeren]
[Indenrigsministeren]
[Boligministeren]
[Ministeren for offentlige arbejder]
[Birgith Mogensen]
[Undervisningsministeren]
[Indenrigsministeren]
[Mølgaard]
[Mimi Jakobsen]
[Undervisningsministeren]
[Finansministeren]
[Undervisningsministeren]
[Udenrigsministeren]
[Flygaard]
[Flygaard]
[Flygaard]
[Flygaard]
[Finansministeren]
[Finansministeren]
[Finansm

[Tang Sørensen]
[Janne Normann]
[Erhard Jakobsen]
[Erhard Jakobsen]
[Erhard Jakobsen]
[Arne Bjerregaard]
[Alfred Hansen]
[Alfred Hansen]
[Landbrugsministeren]
[Landbrugsministeren]
[Stavad]
[Grove]
[Tang Sørensen]
[Ervin Jensen]
[Grove]
[Bjørn Poulsen]
[Janne Normann]
[Landbrugsministeren]
[Brixtofte]
[Socialministeren]
[Socialministeren]
[Socialministeren]
[Brixtofte]
[Brixtofte]
[Brixtofte]
[Inge Fischer Møller]
[Inge Fischer Møller]
[Nathalie Lind]
[Nathalie Lind]
[Nathalie Lind]
[Agnete Laustsen]
[Agnete Laustsen]
[Pilgaard Andersen]
[Pilgaard Andersen]
[Birthe Nielsen]
[Birthe Nielsen]
[Birthe Nielsen]
[Aase Olesen]
[Aase Olesen]
[Arne Melchior]
[Arne Melchior]
[Arne Melchior]
[Anne Grete Holmsgård]
[Anne Grete Holmsgård]
[Anne Grete Holmsgård]
[Inge Krogh]
[Flygaard]
[Flygaard]
[Ingerlise Koefoed]
[Erik B. Smith]
[Taanquist]
[Agnete Laustsen]
[Nathalie Lind]
[Socialministeren]
[Socialministeren]
[Brixtofte]
[Nathalie Lind]
[Erk B. Smith]
[Birthe Nielsen]
[Arne Melchior]
[Socialmi

[Rahbæk Møller]
[Bollmann]
[Wilhjelm]
[Arne Bjerregaard]
[Flygaard]
[Flygaard]
[Ministeren for skatter og afgifter]
[Ervin Jensen]
[Askjær Jørgensen]
[Kurt Hansen]
[Arne Bjerregaard]
[Dyremose]
[Dyremose]
[Janne Normann]
[Økonomiministeren]
[Stavad]
[Ellemann-Jensen]
[Askjær Jørgensen]
[Ib Christensen]
[Rahbæk Møller]
[Stavad]
[Økonomiministeren]
[Taanquist]
[Aase Olesen]
[Ivar Hansen]
[Indenrigsministeren]
[Ministeren for skatter og afgifter]
[Hans Hækkerup]
[Annelise Gotfredsen]
[Maisted]
[Maisted]
[Kalnæs]
[Kalnæs]
[Kalnæs]
[Lone Dybkjær]
[Lone Dybkjær]
[Waldorff]
[Wilhjelm]
[Waldorff]
[Første næstformand]
[Erhard Jakobsen]
[Waldorff]
[Waldorff]
[Waldorff]
[Poul H. Møller]
[Dyremose]
[Finansministeren]
[Christophersen]
[Statsministeren]
[Anne Grete Holmsgård]
[Anne Grete Holmsgård]
[Undervisningsministeren]
[Undervisningsministeren]
[Undervisningsministeren]
[Ole Bernt Henriksen]
[Ministeren for skatter og afgifter]
[Erhard Jakobsen]
[Nathalie Lind]
[Boligministeren]
[Kalnæs]
[Miljø

[Glistrup]
[Ministeren for kulturelle anliggender]
[Øllgaard]
[Flemming Jensen]
[Marianne Bentsen-Pedersen]
[Steffensen]
[Molgaard]
[Molgaard]
[Søren Hansen]
[Eva Møller]
[Mimi Jakobsen]
[Bertel Haarder]
[Ole Vig Jensen]
[Bertel Haarder]
[Formanden]
[Formanden]
[Lawaetz]
[Alfred Hansen]
[Undervisningsministeren]
[Formanden]
[Ministeren for skatter og afgifter]
[Ministeren for skatter og afgifter]
[Indenrigsministeren]
[Indenrigsministeren]
[Ministeren for offentlige arbejder]
[Ministeren for offentlige arbejder]
[Steffensen]
[Justitsministeren]
[Nymann]
[Ministeren for kulturelle anliggender]
[Lawaetz]
[Dohrmann]
[Wilhjelm]
[Wilhjelm]
[Wilhjelm]
[Wilhjelm]
[Tyroll]
[Industriministeren]
[Socialministeren]
[Landbrugsministeren]
[Udenrigsministeren]
[Industriministeren]
[Miljøministeren]
[Grove]
[Hans Jørgen Holm]
[Lilli Gyldenkilde]
[Tyroll]
[Ib Christensen]
[Tyroll]
[Indenrigsministeren]
[Formanden]
[Arbejdsministeren]
[Arbejdsministeren]
[Stavad]
[Niels Anker Kofoed]
[Bjørn Poulsen]
[A

[Wilhjelm]
[Justitsministeren]
[Justitsministeren]
[Justitsministeren]
[Pilgaard Andersen]
[Agnete Laustsen]
[Anne Grete Holmsgård]
[Hans Jorgen Holm]
[Grethe Fenger Møller]
[Voigt]
[Bolvig]
[Voigt]
[Arne Melchior]
[Lilli Gyldenkilde]
[Tyroll]
[Alfred Hansen]
[Alfred Hansen]
[Poul H. Moller]
[Poul H. Møller]
[Arbejdsministeren]
[Arbejdsministeren]
[Hans Jørgen Holm]
[Grethe Fenger Meiler]
[Poul H. Meiler]
[K. B. Andersen]
[K. B. Andersen]
[Industriministeren]
[Kjelgaard]
[Rahbæk Møller]
[Brixtofte]
[Dræbye]
[Kalnæs]
[Keld Andersen]
[Mimi Jakobsen]
[Mimi Jakobsen]
[Dræbye]
[Anne Grete Holmsgård]
[Boligministeren]
[Kalnæs]
[Keld Andersen]
[Dræbye]
[Anne Grete Holmsgård]
[Boligministeren]
[Tredje næstformand]
[Formanden]
[Tyroll]
[Tyroll]
[Formanden]
[Hagen Hagensen]
[Flygaard]
[Ervin Jensen]
[Fogh Rasmussen]
[Fogh Rasmussen]
[Dyremose]
[Dyremose]
[Askjær Jørgensen]
[Industriministeren]
[Dyremose]
[Dyremose]
[Anden næstformand]
[Anden næstformand]
[Enggaard]
[Enggaard]
[Grethe Fenger Møll

[Udenrigsministeren]
[Socialministeren]
[Ministeren for skatter og afgifter]
[Forsvarsministeren]
[Miljøministeren]
[Anders Poulsen]
[Ministeren for skatter og afgifter]
[Gert Petersen]
[Socialministeren]
[Undervisningsministeren]
[Arbejdsministeren]
[Birthe Nielsen]
[Ellemann-Jensen]
[Økonomiministeren]
[Økonomiministeren]
[Boligministeren]
[Voigt]
[Voigt]
[Ministeren for skatter og afgifter]
[Justitsministeren]
[Margrete Auken]
[Dohrmann]
[Socialministeren]


Her findes de forskellige navne fra marginerne, som skal sorteres fra.

## Sammensætning af det ovenstående

I det følgende vil jeg sætte alt det ovenstående sammen, med det formål at udtrække folketingstaler, samt hvem der taler, og sortere marginer fra.

In [182]:
indhold = FT_data
talere = []
tale = []
parti = []
count_folketing = 0
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"]
for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                    talere.append(sentence)
                    parti.append(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0])
                    tale.append("")
                    count_folketing += 1
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\([\w ]*\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence

Jeg laver nu en funktion ud af dette:

In [192]:
def FT_taler(indhold):
    talere = []
    tale = []
    parti = []
    count_folketing = 0
    Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"]
    for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                    talere.append(sentence)
                    parti.append(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0])
                    tale.append("")
                    count_folketing += 1
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\([\w ]*\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence
    return tale, talere, parti

Ovenstående funktion danner basis for formatering af data til senere analyse.